<a href="https://colab.research.google.com/github/Vaibhav9029/mlops2025w_142502033/blob/main/BackpropScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
class Layer:
  def forward(self,x):
    raise NotImplementedError

  def backward(self,grad_output):
    raise NotImplementedError

  def step(self,lr):
    pass

In [ ]:

class Linear:
  def __init__(self,in_features,out_features):
    self.W = torch.randn(in_features,out_features) *0.01
    self.b = torch.zeros(1,out_features)

  def forward(self,x):
    self.x=x
    return x @ self.W + self.b

  def backward(self,grad_output):
    self.dw=self.x.T @ grad_output / self.x.shape[0]
    self.db=grad_output.mean(dim=0,keepdim=True)
    grad_input = grad_output @ self.W.T
    return grad_input

  def step(self,lr):
    self.W-= lr*self.dw
    self.b-= lr*self.db

In [ ]:
class ReLU(Layer):
  def forward(self,x):
    self.mask= x>0
    return x*self.mask

  def backward(self, grad_output):
    return grad_output*self.mask

In [ ]:
class Sigmoid(Layer):
  def forward(self,x):
    self.out = 1/(1+torch.exp(-x))
    return self.out

  def backward(self,grad_output):
    return grad_output*self.out*(1-self.out)

In [ ]:
class BCELoss:
  def forward(self,y_pred,y_true):
    self.y_pred=y_pred
    self.y_true=y_true
    eps = 1e-8
    loss = -(y_true*torch.log(y_pred+eps)+(1-y_true)*torch.log(1-y_pred+eps))
    return loss.mean()

  def backward(self):
    return (self.y_pred - self.y_true) / self.y_true.shape[0]


In [ ]:
class NeuralNetwork:
  def __init__(self,layers):
    self.layers=layers

  def forward(self,x):
    for layer in self.layers:
      x = layer.forward(x)
    return x

  def backward(self,grad):
    for layer in reversed(self.layers):
      grad = layer.backward(grad)

  def step(self,lr):
    for layer in self.layers:
      layer.step(lr)

In [ ]:
# Dataset
X = torch.tensor([[0., 0.],
                  [0., 1.],
                  [1., 0.],
                  [1., 1.]])

y = torch.tensor([[0.],
                  [1.],
                  [1.],
                  [0.]])


In [ ]:
# Model definition (ANY depth works)
model = NeuralNetwork([
    Linear(2, 8),
    ReLU(),
    Linear(8, 8),
    ReLU(),
    Linear(8, 1),
    Sigmoid()
])

loss_fn = BCELoss()
lr = 0.1


In [ ]:
# Training loop
for epoch in range(5000):

    # Forward pass
    y_pred = model.forward(X)

    # Loss
    loss = loss_fn.forward(y_pred, y)

    # Backprop
    grad = loss_fn.backward()
    model.backward(grad)

    # Update
    model.step(lr)

    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 0.6931
Epoch 500, Loss: 0.6931
Epoch 1000, Loss: 0.6931
Epoch 1500, Loss: 0.6931
Epoch 2000, Loss: 0.6931
Epoch 2500, Loss: 0.6931
Epoch 3000, Loss: 0.6931
Epoch 3500, Loss: 0.6931
Epoch 4000, Loss: 0.6931
Epoch 4500, Loss: 0.6931


In [ ]:
with torch.no_grad():
    preds = (model.forward(X) > 0.5).float()
    print("Predictions:\n", preds)
    print("Targets:\n", y)


Predictions:
 tensor([[0.],
        [1.],
        [0.],
        [0.]])
Targets:
 tensor([[0.],
        [1.],
        [1.],
        [0.]])
